In [1]:
from __future__ import division
import numpy as np
import copy
# from .t_astron import t_astron
from ttide.t_astron import t_astron
import os.path as path
from scipy.io.netcdf import netcdf_file as nopen
import datetime
from ttide.time import date2num

/tmp/ipykernel_7112/1535637445.py:7: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  from scipy.io.netcdf import netcdf_file as nopen


In [3]:

_base_dir = path.join("/home/tzw-linux/software/ttide_py-master/ttide", 'data')
has_const = path.exists(path.join(_base_dir, 't_constituents_const.nc'))
has_sat = path.exists(path.join(_base_dir, 't_constituents_sat.nc'))
has_shallow = path.exists(path.join(_base_dir, 't_constituents_shallow.nc'))

if (has_const and has_sat and has_shallow):
    _const = {}
    _sat = {}
    _shallow = {}

    ncid = nopen(path.join(_base_dir,
                           't_constituents_const.nc'), 'r', mmap=False)
    for key in ncid.variables.keys():
        _const[key] = ncid.variables[key].data
    ncid.close()

    ncid = nopen(path.join(_base_dir,
                           't_constituents_sat.nc'), 'r', mmap=False)
    for key in ncid.variables.keys():
        _sat[key] = ncid.variables[key].data
    ncid.close()

    ncid = nopen(path.join(_base_dir,
                           't_constituents_shallow.nc'), 'r', mmap=False)
    for key in ncid.variables.keys():
        _shallow[key] = ncid.variables[key].data
    ncid.close()

    # Correct issues with name strings
    _const['name'] = np.array([b''.join([s for s in arr])
                               for arr in _const['name']])

    _const['kmpr'] = np.array([b''.join([s for s in arr])
                               for arr in _const['kmpr']])

else:
    print('You do not have t_constituents_*.npy ' +
          'check that package installation is correct.')
    _const = {}
    _sat = {}
    _shallow = {}


In [4]:
ctime = datetime.datetime(2015,1,1,0,0,0)

In [6]:
const = copy.deepcopy(_const)
sat = copy.deepcopy(_sat)
shallow = copy.deepcopy(_shallow)

astro, ader = t_astron(date2num(ctime))

ii = np.isfinite(const['ishallow'])
const['freq'][~ii] = np.dot(const['doodson'][~ii, :], ader) / 24

shallow_m1 = const['ishallow'].astype(int) -1
iname_m1 = shallow['iname'].astype(int) -1
range_cache = {n:np.arange(n) for n in range(const['nshallow'].max()+1)}
for k in np.flatnonzero(ii):
    ik = shallow_m1[k] + range_cache[const['nshallow'][k]]
    # print(ik)
    const['freq'][k] = const['freq'][iname_m1[ik]].dot(shallow['coef'][ik])


In [67]:
const["doodson"][0:10,:]

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0., -1.],
       [ 0.,  0.,  2.,  0.,  0.,  0.],
       [ 0.,  1., -2.,  1.,  0.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.],
       [ 0.,  2., -2.,  0.,  0.,  0.],
       [ 0.,  2.,  0.,  0.,  0.,  0.],
       [ 1., -4.,  2.,  1.,  0.,  0.],
       [ 1., -3.,  0.,  2.,  0.,  0.],
       [ 1., -3.,  2.,  0.,  0.,  0.]])